In [3]:
pip install pickle

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [4]:
# Importing the libraries


import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import pickle

ModuleNotFoundError: No module named 'mediapipe'

In [2]:
mp_drawing = mp.solutions.drawing_utils     # Connecting Keypoints Visuals
mp_pose = mp.solutions.pose                 # Keypoint detection model

# Landmark Positions
1. We will need the angle at the elbow to count the number of curls and also to determine incorrect curl during the exercise. We will figure it out using the coordinates of shoulder, elbow and wrist.

![Keypoints](https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png)
* Image Source: https://google.github.io/mediapipe/solutions/pose.html

* We are concerned with only three keypoints that are shoulder, elbow and wrist of the same hand; be it either left or right.

In [3]:
print("Total number of landmarks = {}".format(len(results.pose_landmarks.landmark)))
results.pose_landmarks.landmark[0:2] # Output of first two landmarks in the container

NameError: name 'results' is not defined

In [4]:
# Sequential mapping of the landmarks with human body parts

count = 0
for i in mp_pose.PoseLandmark:
    print(count, i)
    count += 1

0 PoseLandmark.NOSE
1 PoseLandmark.LEFT_EYE_INNER
2 PoseLandmark.LEFT_EYE
3 PoseLandmark.LEFT_EYE_OUTER
4 PoseLandmark.RIGHT_EYE_INNER
5 PoseLandmark.RIGHT_EYE
6 PoseLandmark.RIGHT_EYE_OUTER
7 PoseLandmark.LEFT_EAR
8 PoseLandmark.RIGHT_EAR
9 PoseLandmark.MOUTH_LEFT
10 PoseLandmark.MOUTH_RIGHT
11 PoseLandmark.LEFT_SHOULDER
12 PoseLandmark.RIGHT_SHOULDER
13 PoseLandmark.LEFT_ELBOW
14 PoseLandmark.RIGHT_ELBOW
15 PoseLandmark.LEFT_WRIST
16 PoseLandmark.RIGHT_WRIST
17 PoseLandmark.LEFT_PINKY
18 PoseLandmark.RIGHT_PINKY
19 PoseLandmark.LEFT_INDEX
20 PoseLandmark.RIGHT_INDEX
21 PoseLandmark.LEFT_THUMB
22 PoseLandmark.RIGHT_THUMB
23 PoseLandmark.LEFT_HIP
24 PoseLandmark.RIGHT_HIP
25 PoseLandmark.LEFT_KNEE
26 PoseLandmark.RIGHT_KNEE
27 PoseLandmark.LEFT_ANKLE
28 PoseLandmark.RIGHT_ANKLE
29 PoseLandmark.LEFT_HEEL
30 PoseLandmark.RIGHT_HEEL
31 PoseLandmark.LEFT_FOOT_INDEX
32 PoseLandmark.RIGHT_FOOT_INDEX


In [5]:
def calc_angle(a,b,c): # 3D points
    ''' Arguments:
        a,b,c -- Values (x,y,z, visibility) of the three points a, b and c which will be used to calculate the
                vectors ab and bc where 'b' will be 'elbow', 'a' will be shoulder and 'c' will be wrist.
        
        Returns:
        theta : Angle in degress between the lines joined by coordinates (a,b) and (b,c)
    '''
    a = np.array([a.x, a.y])#, a.z])    # Reduce 3D point to 2D
    b = np.array([b.x, b.y])#, b.z])    # Reduce 3D point to 2D
    c = np.array([c.x, c.y])#, c.z])    # Reduce 3D point to 2D

    ab = np.subtract(a, b)
    bc = np.subtract(b, c)
    
    theta = np.arccos(np.dot(ab, bc) / np.multiply(np.linalg.norm(ab), np.linalg.norm(bc)))     # A.B = |A||B|cos(x) where x is the angle b/w A and B
    theta = 180 - 180 * theta / 3.14    # Convert radians to degrees
    return np.round(theta, 2)

In [ ]:
flag = None     # Flag which stores hand position(Either UP or DOWN)
count = 0       # Storage for count of bicep curls


cap = cv2.VideoCapture(0)
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5) # Lnadmark detection model instance
while cap.isOpened():
    _, frame = cap.read()

    # BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)      # Convert BGR frame to RGB
    image.flags.writeable = False
    
    # Make Detections
    results = pose.process(image)                       # Get landmarks of the object in frame from the model

    # Back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      # Convert RGB back to BGR

    try:
        # Extract Landmarks
        landmarks = results.pose_landmarks.landmark
        shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]


        # Calculate angle
        angle = calc_angle(shoulder, elbow, wrist)      #  Get angle 


        # Visualize angle
        cv2.putText(image,\
                str(angle), \
                    tuple(np.multiply([elbow.x, elbow.y], [640,480]).astype(int)),\
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)
    
        # Counter 
        if angle > 160:
            flag = 'down'
        if angle < 40 and flag=='down':
            count += 1
            flag = 'up'
            print("Count: {}, Flag: {}".format(count, flag))
        

    except:
        pass


    # Setup Status Box
    cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
    cv2.putText(image, str(count), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


    # Render Detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)


    cv2.imshow('MediaPipe feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
model_dir = "model"
tf.saved_model.save(model, model_dir)

In [1]:
filename ="biceps_pose"
pickle.dump(model,open(filename,'wb'))

NameError: name 'pickle' is not defined

# References:

1. Pose Classification [https://google.github.io/mediapipe/solutions/pose_classification.html]

2. MediaPipePoseEstimation [https://github.com/nicknochnack/MediaPipePoseEstimation]

3. Guide to Human Pose Estimation with Deep Learning[https://nanonets.com/blog/human-pose-estimation-2d-guide/]

4. Real-time Human Pose Estimation in the Browser [https://blog.tensorflow.org/2018/05/real-time-human-pose-estimation-in.html]